# Ape Minders: Trading Cryptocurrency on NFT Signals

By Andreas Michael, Michael Wander, Daria Vasileva, and Amit Kumar

## Introduction

The market of non-fungible tokens, or NFT's for short, has boomed over the past two years, with interest and trading at an all-time high. Although the hype cannot last forever, the effects of NFT's on cryptocurrency as a whole should not be dismissed. The most popular and most expensive NFT collections are generally based on the Ethereum blockchain, mainly through Polygon on OpensSea, but marketplaces on other blockchains now exist for the trading and minting of NFT's, including Solana, Cardano, Avalanche, Decentraland, and others. With this in mind, we aim to trade various cryptocurrencies, from large to small-cap, which are directly or indirectly related to the trading of NFT's, on signals produced from NFT data, such as number of sales, active wallets, buyers, and sales in USD, as well as sentiment and polarity data of text posts related to NFT's. We presume that NFT related data correlates well with our chosen universe of cryptocurrencies, which should allow for an effective trading strategy.  

## Market Universe (Instruments, Benchmark, Exchange)

## NFT Features: Quantitative Data

## NFT Features: Alternative Data

## Regression Models for Predicted Returns

## Trading Algorithm

## Portfolio Evaluation

## Conclusion